In [15]:
from dash import Dash, html, dcc, dash_table, Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd

In [16]:
app=Dash(__name__)

education=(pd.read_csv('states_all.csv')
.iloc[:, 1:]
.assign(
    Avg_Math=lambda x:x[['AVG_MATH_4_SCORE','AVG_MATH_8_SCORE']].mean(axis=1),
    Avg_Reading=lambda x:x[['AVG_READING_4_SCORE','AVG_READING_8_SCORE']].mean(axis=1)
))

all_options={
    'By Subject':[
        {'label':'Math', 'value':'Avg_Math'},
        {'label':'Rading', 'value':'Avg_Reading'}
    ],
    'By Grade and Subject':[
        {'label':'4th Grade Math','value':'AVG_MATH_4_SCORE'},
        {'label':'8th Grade Math','value':'AVG_MATH_8_SCORE'},
        {'label':'4th Grade Reading','value':'AVG_READING_4_SCORE'},
        {'label':'8th Grade Reading','value':'AVG_READING_8_SCORE'},
    ]
}

app.layout=html.Div([
    dcc.RadioItems(
        id='report-type-radio',
        options=['By Subject','By Grade and Subject'],
        value='By Grade and Subject'
    ),
    dcc.RadioItems(id='metric-radio'),
    dcc.Graph(id='metric-bar')
])

@app.callback(Output('metric-radio','options'),Input('report-type-radio','value'))
def set_metrics_options(selected_report):
    return all_options[selected_report]

@app.callback(Output('metric-bar','figure'),Input('metric-radio','value'))
def plot_bar(metric):
    if not metric:
        raise PreventUpdate
    figure=px.bar((
        education
        .groupby('STATE',as_index=False)
        .agg({metric:'mean'})
        .sort_values(metric,ascending=False)
    ),
    x='STATE',
    y=metric)

    return figure

app.run_server(debug=True, port=8051)